In [2]:
import csv
import geopandas as gp
import pandas as pd

In [3]:
#Process presidential votes
state = 'PA'
democrat = 'trump'
republican = 'clinton'
STATE_FIPS = '42'

president = {}
reader = csv.reader(open('./election/2016-precinct-president.csv', 'r'))
for row in reader:
    if row[4] != state:
        continue
    #democr, republican
    id = row[13]
    
    id = id.zfill(5)
    county_fips = row[8]
    id = county_fips + id    
       
    
    if id in president.keys():
        votes = president[id]
    else:
        votes = [0, 0]
    if row[15] == democrat:
        votes[0] += int(row[21])
    if row[15] == republican:
        votes[1] += int(row[21])
    president[id] = votes
    
print (president)
        
    

{'4200100010': [274, 119], '4200100020': [235, 143], '4200100030': [153, 83], '4200100040': [816, 257], '4200100050': [314, 148], '4200100060': [519, 191], '4200100070': [882, 308], '4200100080': [797, 345], '4200100085': [625, 283], '4200100090': [1012, 431], '4200100100': [1097, 415], '4200100110': [584, 507], '4200100120': [587, 209], '4200100130': [563, 397], '4200100210': [325, 491], '4200100310': [1013, 297], '4200100320': [501, 205], '4200100330': [701, 203], '4200100410': [524, 191], '4200100420': [403, 250], '4200100425': [772, 303], '4200100510': [114, 65], '4200100170': [811, 315], '4200100230': [327, 855], '4200100240': [320, 435], '4200100200': [1028, 255], '4200100150': [523, 217], '4200100140': [514, 428], '4200100160': [175, 102], '4200100190': [356, 152], '4200100180': [769, 264], '4200100260': [1006, 278], '4200100350': [393, 189], '4200100360': [301, 150], '4200100370': [1136, 384], '4200100440': [1067, 393], '4200100450': [793, 271], '4200100500': [1155, 311], '4200

In [4]:
file = "./PA/PA_new.geojson"
precincts = gp.read_file(file)
precincts.head()

,type,centerX,centerY,area,neighbors,canonical,id,geometry
0,ORIGINAL,41.157814,-80.479783,0.004278359631999892,"4207300780, 4207300790, 4208500950, 4208501040...",PA Mercer 00960,4208500960,"POLYGON ((-80.51922 41.18555, -80.51905 41.185..."
1,ORIGINAL,41.680613,-80.04085000000001,0.0005506003439999464,"4203900150, 4203900330, 4203900340, 4203900660",PA Crawford 00040,4203900040,"POLYGON ((-80.05844 41.68391, -80.05597 41.684..."
2,ORIGINAL,41.743054,-79.840985,0.007933951458499875,"4203900030, 4203900070, 4203900400, 4203900360...",PA Crawford 00010,4203900010,"POLYGON ((-79.90979 41.77394, -79.90930 41.773..."
3,ORIGINAL,41.803153,-80.462641,0.01028330178850014,"4203900090, 4203900420, 4204900040",PA Crawford 00020,4203900020,"POLYGON ((-80.51935 41.84956, -80.51544 41.849..."
4,ORIGINAL,41.812444,-79.836173,0.01073166413550007,"4203900010, 4203900400, 4203900360, 4203900350...",PA Crawford 00030,4203900030,"POLYGON ((-79.90998 41.85069, -79.90995 41.850..."


In [14]:
president_votes = precincts.copy()
president_votes = president_votes.drop(columns = ['type', 'centerX', 'centerY', 'canonical', 'geometry', 'neighbors', 'area'])

president_votes.insert(1, 'democrat', 0)
president_votes.insert(2, 'republican', 0)

president_votes.head()

,id,democrat,republican
0,4208500960,0,0
1,4203900040,0,0
2,4203900010,0,0
3,4203900020,0,0
4,4203900030,0,0


In [15]:
notFound = []
for i, row in president_votes.iterrows():
    id = str(row['id'])
    if id in president.keys():
        president_votes.at[i, 'democrat'] = president[id][0]
        president_votes.at[i, 'republican'] = president[id][1]
        #row['democrat'] = president[id][0]
        #row['republican'] = president[id][1]
        
    else:
        notFound.append(id)
    #print(president_votes.at[i, 'democrat'])

In [18]:
print(notFound)
print(president_votes[president_votes['republican'] == 0 ].shape)
#for i, row in president_votes.iterrows():
    #if row['democrat'] == 0 or row['republican'] == 0:
       # print(i)

['421090PR26', '421090PR28', '4210900190', '4210900260', '4210900270', '4201300060', '4201300200', '4201300270', '4201300280', '4201300310', '4201300360', '4201300440', '4206300685', '4210300021', '4210300051', '4207300285', '4207300600', '4207300650', '4207300730', '4207300740', '4207300890', '4207300040', '4207300060', '4204700130', '4207301020', '4207301030', '4207300180', '4204700270', '4204700280', '4204700290', '4200900380', '4200900360', '4208700170', '4208700190', '4208700200', '4208700220', '4208700240', '4200900070', '4201100362', '4201100355', '4201100768', '4201100785', '4201101512', '4201101495', '4201100195', '4201102121', '4201100360', '4201101690', '4201100640', '420110210A', '420110050A', '420110040A', '420110040B', '420111510B', '4201101511', '4201100540', '420110050B', '4201101080', '420111140B', '420110400A', '4201101140', '4201101180', '4201100940', '4201101790', '4201101700', '4201101480', '4201101710', '4201100665', '420111450A', '4201100386', '4201101300', '4201

In [19]:
pd.DataFrame(president_votes).to_json('./election/PA_president2016.json')

In [41]:
#PROCESS CONGRESSIONAL VOTES 2016
#SENATE
STATE = 'PA'
DEMOCRATIC = 'mcginty'
REPUBLICAN = 'toomey'
senate2016 = {}

reader = csv.reader(open('./election/2016-precinct-senate.csv', 'r'))
for row in reader:
    if row[4] != STATE:
        continue
    #democr, republican
    id = row[13]
    id = id.zfill(4)
    county_fips = row[8]
    id = county_fips + id    
    
    if id in senate2016.keys():
        votes = senate2016[id]
    else:
        votes = [0, 0]
        
    if row[15] == DEMOCRATIC:
        votes[0] += int(row[21])
    if row[15] == REPUBLICAN:
        votes[1] += int(row[21])
    senate2016[id] = votes
    
print (senate2016)

{'420010010': [128, 256], '420010020': [141, 232], '420010030': [83, 159], '420010040': [289, 745], '420010050': [150, 306], '420010060': [216, 480], '420010070': [301, 869], '420010080': [364, 752], '420010085': [277, 630], '420010090': [449, 974], '420010100': [435, 1063], '420010110': [495, 599], '420010120': [222, 557], '420010130': [384, 589], '420010210': [494, 322], '420010310': [324, 1002], '420010320': [214, 481], '420010330': [227, 679], '420010410': [214, 501], '420010420': [266, 380], '420010425': [336, 729], '420010510': [71, 105], '420010170': [326, 798], '420010230': [772, 409], '420010240': [422, 356], '420010200': [268, 956], '420010150': [228, 494], '420010140': [424, 540], '420010160': [100, 173], '420010190': [152, 358], '420010180': [284, 725], '420010260': [295, 969], '420010350': [224, 339], '420010360': [159, 290], '420010370': [394, 1103], '420010440': [411, 1035], '420010450': [295, 726], '420010500': [340, 1081], '420010390': [289, 725], '420010280': [145, 37

In [42]:
#HOUSE
STATE = 'PA'
DEMOCRAT = ('balchunis' , 'boyle', 'brady', 'burkholder', 'cartwright', 'daugherty', 'doyle', 'evans',
            'halvorson', 'hartman', 'marsicano', 'mcclelland', 'molesevich', 'parrish', 'santarsiero', 'taylor',
           )
REPUBLICAN = ('barletta', 'connolly', 'costello', 'dent', 'fitzpatrick', 'jones', 'kelly', 'marino', 'mcallister',
              'meehan', 'murphy', 'perry', 'rothfus', 'shuster', 'smucker', 'thompson', 'williams', 
             )

house2016 = {}

reader = csv.reader(open('./election/2016-precinct-house.csv', 'r', encoding = "utf-8"))
for row in reader:
    if row[4] != STATE:
        continue
    #democrat, republican
    id = row[13]
    
    id = id.zfill(4)
    county_fips = row[8]
    id = county_fips + id    
    
    if id in house2016.keys():
        votes = house2016[id]
    else:
        votes = [0, 0]
        
    if row[15] in DEMOCRAT:
        votes[0] += int(row[21])
    elif row[15] in REPUBLICAN:
        votes[1] += int(row[21])
    house2016[id] = votes
    
print (house2016)

{'420010010': [113, 298], '420010020': [123, 268], '420010030': [76, 177], '420010040': [248, 833], '420010050': [134, 346], '420010060': [184, 545], '420010070': [248, 968], '420010080': [328, 839], '420010085': [269, 673], '420010090': [378, 1076], '420010100': [359, 1164], '420010110': [455, 653], '420010120': [186, 611], '420010130': [330, 660], '420010210': [468, 369], '420010310': [273, 1097], '420010320': [187, 531], '420010330': [195, 745], '420010410': [175, 562], '420010420': [257, 430], '420010425': [278, 817], '420010510': [61, 123], '420010170': [292, 857], '420010230': [755, 448], '420010240': [408, 389], '420010200': [223, 1060], '420010150': [184, 572], '420010140': [364, 613], '420010160': [82, 202], '420010190': [145, 386], '420010180': [252, 795], '420010260': [266, 1037], '420010350': [204, 411], '420010360': [137, 327], '420010370': [343, 1210], '420010440': [358, 1131], '420010450': [251, 836], '420010500': [283, 1198], '420010390': [252, 791], '420010280': [134, 

In [43]:
congress_votes = df.copy()
congress_votes = congress_votes.drop(columns = ['type', 'centerX', 'centerY', 'canonical', 'geometry'])

congress_votes.insert(1, 'democrat', 0)
congress_votes.insert(2, 'republican', 0)

congress_votes.head()

,id,democrat,republican
0,420010040,0,0
1,420010010,0,0
2,420010260,0,0
3,420010500,0,0
4,420010100,0,0


In [44]:
notFound = []
for i, row in congress_votes.iterrows():
    id = str(row['id'])
    
    if id in senate2016.keys():       
        congress_votes.at[i, 'democrat'] = senate2016[id][0]
        congress_votes.at[i, 'republican'] = senate2016[id][1]
    elif id in house2016.keys():
        democrats = congress_votes.at[i, 'democrat']
        congress_votes.at[i, 'democrat'] = democrats + house2016[id][0]
        republicans = congress_votes.at[i, 'republican']
        congress_votes.at[i, 'republican'] = republicans + house2016[id][1]
    else:
        notFound.append(id)

In [47]:
congress_votes.head()
congress_votes[congress_votes['republican'] == 0].shape

(179, 3)

In [48]:
pd.DataFrame(congress_votes).to_json("./election/PA_congress2016.json")

In [50]:
reader = csv.reader(open('./PA_county.csv', 'r'))
d = {}
for row in reader:
    #id = row['County FIPS']
    value = row[3]
    key = row[5]
    d[key] = value
    
print (d)

{'Short County Name': 'County FIPS Code', 'Crawford': '039', 'Luzerne ': '079', 'Lehigh ': '077', 'McKean': '083', 'Monroe': '089', 'Centre': '027', 'Cumberland': '041', 'Dauphin': '043', 'Delaware': '045', 'Fayette': '051', 'Lawrence ': '073', 'Lycoming': '081', 'Montgomery': '091', 'Montour': '093', 'Northampton': '095', 'Elk': '047', 'Erie': '049', 'Lebanon ': '075', 'Beaver': '007', 'Franklin': '055', 'Fulton': '057', 'Lancaster': '071', 'Mercer': '085', 'Greene': '059', 'Huntingdon': '061', 'Indiana': '063', 'Jefferson': '065', 'Juniata': '067', 'Lackawanna': '069', 'Mifflin': '087', 'Northumberland': '097', 'Perry': '099', 'Philadelphia': '101', 'Pike': '103', 'Potter': '105', 'Schuylkill': '107', 'Snyder': '109', 'Somerset': '111', 'Sullivan': '113', 'Susquehanna': '115', 'Tioga': '117', 'Union': '119', 'Venango': '121', 'Warren': '123', 'Washington': '125', 'Wayne': '127', 'Westmoreland': '129', 'Wyoming': '131', 'York': '133', 'Butler': '019', 'Cambria': '021', 'Cameron': '023

In [52]:
### CONGRESS 2018 
STATE_FIPS = '42'
STATE = 'PA'
congress2018 = {}
reader = csv.reader(open('./election/2018_precinct_congress_PA.csv', 'r', encoding = "utf-8"))
for row in reader:
    if row[15] != STATE:
        continue
    #democrat, republican
    id = row[0].zfill(4)
    if row[6] in d.keys():
        id = STATE_FIPS + d[row[6]] + id
        #print(id)
    else:        
        continue
    if id in congress2018.keys():
        votes = congress2018[id]
    else:
        votes = [0, 0]
        
    if row[2] == 'democrat':
        votes[0] += int(row[4])
    elif row[2] == 'republican':
        votes[1] += int(row[4])
    congress2018[id] = votes
    
print (congress2018)

{'420030010': [908, 845], '420030020': [426, 325], '420030030': [558, 215], '420030040': [818, 332], '420030050': [857, 300], '420030060': [612, 208], '420030070': [362, 185], '420030080': [242, 142], '420030090': [375, 188], '420030100': [399, 174], '420030110': [677, 422], '420030120': [555, 301], '420030130': [488, 123], '420030140': [955, 207], '420030150': [693, 158], '420030160': [609, 165], '420030170': [675, 169], '420030180': [566, 171], '420030190': [619, 200], '420030200': [614, 151], '420030210': [663, 147], '420030225': [534, 95], '420030235': [400, 135], '420030240': [554, 144], '420030250': [573, 177], '420030260': [504, 145], '420030270': [706, 165], '420030280': [741, 186], '420030290': [589, 168], '420030300': [841, 183], '420030310': [761, 743], '420030320': [781, 290], '420030330': [796, 278], '420030340': [752, 286], '420030350': [786, 300], '420030360': [792, 315], '420030370': [713, 338], '420030380': [556, 172], '420030390': [905, 447], '420030400': [274, 220], 

In [54]:
congress_votes = df.copy()
congress_votes = congress_votes.drop(columns = ['type', 'centerX', 'centerY', 'canonical', 'geometry'])

congress_votes.insert(1, 'democrat', 0)
congress_votes.insert(2, 'republican', 0)

congress_votes.head()

,id,democrat,republican
0,420010040,0,0
1,420010010,0,0
2,420010260,0,0
3,420010500,0,0
4,420010100,0,0


In [55]:
notFound = []
for i, row in congress_votes.iterrows():
    id = str(row['id'])
    
    if id in congress2018.keys():
        congress_votes.at[i, 'democrat'] = congress2018[id][0]
        congress_votes.at[i, 'republican'] = congress2018[id][1]        
    else:
        notFound.append(id)

In [56]:
print(len(notFound))
print(congress_votes[congress_votes['republican'] == 0])
congress_votes.head()

842
              id  democrat  republican
105    420039240       438           0
193    420039300       630           0
196    420039330       615           0
284   4200312070       820           0
314    420034000       771           0
...          ...       ...         ...
8659   421270180         0           0
8684   421291250         0           0
8962   421292270         0           0
8964   421292270         0           0
8984   421310180         0           0

[894 rows x 3 columns]


,id,democrat,republican
0,420010040,541,1206
1,420010010,228,384
2,420010260,577,1504
3,420010500,635,1818
4,420010100,898,1784


In [57]:
pd.DataFrame(congress_votes).to_json("./election/PA_congress2018.json")